In [1]:
from rlcube.models.models import DNN
from rlcube.envs.cube2 import Cube2Env
import torch

net = DNN()
net.load("models/model_best.pth")
net.eval()

DNN(
  (fc_in): Linear(in_features=144, out_features=512, bias=True)
  (residual_blocks): ModuleList(
    (0-3): 4 x ResidualBlock(
      (ln1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=512, out_features=1024, bias=True)
      (ln2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (fc2): Linear(in_features=1024, out_features=512, bias=True)
    )
  )
  (fc_value): Sequential(
    (0): Linear(in_features=512, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
  (fc_policy): Sequential(
    (0): Linear(in_features=512, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=12, bias=True)
  )
)

In [16]:
import numpy as np

env = Cube2Env()

actions = []
obs = []
for _ in range(10):
    action = env.action_space.sample()
    actions.append(action.item())
    env.step(action)
    obs.append(env.obs())

obs = torch.tensor(np.array(obs), dtype=torch.float32)
values, policies = net(obs)
print(actions)
print(values)

[2, 3, 7, 6, 8, 6, 3, 2, 2, 5]
tensor([[ 1.1924],
        [ 0.0826],
        [ 1.0202],
        [ 0.0826],
        [ 1.1121],
        [-0.0302],
        [-1.5963],
        [-0.0302],
        [-1.3707],
        [-2.4068]], grad_fn=<AddmmBackward0>)


In [18]:
from rlcube.models.search import MonteCarloTree

tree = MonteCarloTree(env.obs(), max_simulations=300)
if tree.is_solved:
    print([action for _, action in tree.solved_path])

 14%|█▍        | 43/300 [00:00<00:02, 127.98it/s]

[4, 3, 7, 11]
